In [1]:
import json
import re 
import copy
from importlib import reload
import pandas as pd


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


## Configuration

This is controlled from `predicament.utils.config`. This tells us: what data is there, and some of the characteristics of that data.

We also have `predicament.utils.file_utils` which controls file, directory and time stamp information.



In [3]:
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import BASE_DATA_FOLDER
from predicament.utils.config import STUDY_DATA_FOLDER
from predicament.utils.config import MOTOR_MOVEMENT_DATA_FOLDER

from predicament.utils.config import DREEM_EEG_CHANNELS


from predicament.utils.config import E4_file_paths
from predicament.utils.config import VG_file_paths
from predicament.utils.config import VG_Hz
from predicament.utils.config import EEG_buffer
from predicament.utils.config import E4_buffer

print(f"BASE_DATA_FOLDER = {BASE_DATA_FOLDER}")
print(f"MOTOR_MOVEMENT_DATA_FOLDER = {MOTOR_MOVEMENT_DATA_FOLDER}")
print(f"E4_file_paths = {E4_file_paths}")


print(f"VG_file_paths = {VG_file_paths}")
print("\tThis is a dictionary of participant ID to edf file subpaths")

print()
print(f"DREEM_EEG_CHANNELS = {DREEM_EEG_CHANNELS}")
print(f"VG_Hz = {VG_Hz}")
print(f"EEG_buffer = {EEG_buffer}")
print(f"E4_buffer = {E4_buffer}")


BASE_DATA_FOLDER = ../data
MOTOR_MOVEMENT_DATA_FOLDER = ../data/EEG-Motor-Movement-Imagery-Dataset/
E4_file_paths = {'VG_01': './VG01/E4_8921_15_44/', 'VG_03': './VG03/E4_9921_12_16/', 'VG_05': './VG05/E4_9921_13_24/', 'VG_06': './VG06/E4_51021_13_33/', 'VG_07': './VG07/E4_51021_15_39/', 'VG_08': './VG08/E4_71021_10_42/', 'VG_09': './VG09/E4_11221_14_46/', 'VG_10': './VG10/E4_31221_11_17/', 'VH_01': './VH01/E4_61021_11_03/', 'VH_02': './VH02/E4_61021_13_59/', 'VH_03': './VH03/E4_11221_11_22/'}
VG_file_paths = {'VG_01': './VG01/8921_15_52.edf', 'VG_02': './VG02/7921_15_18.edf', 'VG_03': './VG03/90921_12_20.edf', 'VG_05': './VG05/90921_13_27.edf', 'VG_06': './VG06/51021_13_40.edf', 'VG_07': './VG07/51021_15_43.edf', 'VG_08': './VG08/71021_10_44.edf', 'VG_09': './VG09/11221_14_59.edf', 'VG_10': './VG10/31221_11_24.edf', 'VH_01': './VH01/61021_11_17.edf', 'VH_02': './VH02/61021_14_3.edf', 'VH_03': './VH03/11221_11_21.edf'}
	This is a dictionary of participant ID to edf file subpaths

DREEM

## Creating a workable set of data objects

We need some timeseries data of EEG traces and some information regarding the human-level events that took place. Then we need to integrate the two.

In [4]:
from Ray import EEG_data

EEG_files = EEG_data.read_all_VG_files()
EEG_files

STUDY_DATA_FOLDER = ../data/CARE_HOME_DATA/
VG_file_path = ../data/CARE_HOME_DATA/./VG01/8921_15_52.edf
Extracting EDF parameters from /home/luke/git/external/predicament/data/CARE_HOME_DATA/VG01/8921_15_52.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
VG_file.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
include_channels=['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
type(data) = <class 'numpy.ndarray'>
data.shape = (13, 561000)
ch_name = EEG Fpz-O1
channel_lookup[ch_name] = 1
ch_name = EEG Fpz-O2
channel_lookup[ch_name] = 2
ch_name = EEG Fpz-F7
channel_lookup[ch_name] = 3
ch_name = EEG F8-F7
channel_lookup[ch_name] = 4
ch_name = EEG F7-01
channel_lookup[ch_name] = 5
ch_name = EEG F8-O2
channel_lookup[ch_name] = 6
ch_name = EEG

EDF file detected
Setting channel info structure...
Creating raw.info structure...
VG_file.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'Positiongram', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
include_channels=['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
type(data) = <class 'numpy.ndarray'>
data.shape = (14, 1074500)
ch_name = EEG Fpz-O1
channel_lookup[ch_name] = 1
ch_name = EEG Fpz-O2
channel_lookup[ch_name] = 2
ch_name = EEG Fpz-F7
channel_lookup[ch_name] = 3
ch_name = EEG F8-F7
channel_lookup[ch_name] = 4
ch_name = EEG F7-01
channel_lookup[ch_name] = 5
ch_name = EEG F8-O2
channel_lookup[ch_name] = 6
ch_name = EEG Fpz-F8
channel_lookup[ch_name] = 7
Successfully loaded VG_10 VG file (EEG data).
STUDY_DATA_FOLDER = ../data/CARE_HOME_DATA/
VG_file_path = ../data/CARE_HOME_DATA/./VH01/61021_11_17.edf
Extracting

{'VG_01': EEG_data_obj(ID:VG_01, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_02': EEG_data_obj(ID:VG_02, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_03': EEG_data_obj(ID:VG_03, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_05': EEG_data_obj(ID:VG_05, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_06': EEG_data_obj(ID:VG_06, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_07': EEG_data_obj(ID:VG_07, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_08': EEG_data_obj(ID:VG_08, channels:['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']),
 'VG_09': EEG_data_o

In [5]:
from Ray.data_load_save import load_info_from_csv
ev_details = load_info_from_csv()

event_details_path = ../data/CARE_HOME_DATA/./event_details.csv
event_details_df.loc['VG_01'] =                  event     start       end  valid
partID                                           
VG_01            setup  15:52:00      None   True
VG_01         baseline  15:52:00      None   True
VG_01      exper_video  16:02:00  16:10:00   True
VG_01   wildlife_video  15:56:00  16:01:00   True
VG_01   familiar_music  16:16:00  16:21:00   True
VG_01      tchaikovsky  16:11:00  16:16:00   True
VG_01     family_inter  16:22:00  16:27:00   True
event = ('VG_01', event       setup
start    15:52:00
end          None
valid        True
Name: VG_01, dtype: object)
event[1]['end'] = None
type(event[1]['end']) = <class 'str'>
partID: end = VG_01:None
(start, end) = ('15:52:00', 'None')
(start_timestamp, end_timestamp) = (1631112720.0, None)
event = ('VG_01', event    baseline
start    15:52:00
end          None
valid        True
Name: VG_01, dtype: object)
event[1]['end'] = None
type(event[1]['en

In [6]:
for partID, EEG in EEG_files.items():
    EEG.set_event_details(ev_details[partID])

In [7]:
EEG_files['VG_01'].event_details
EEG_files['VG_01'].EEG_data


{'EEG Fpz-O1': array([-8.16281376e-05, -5.00000000e-04, -5.00000000e-04, ...,
         7.62951095e-09,  7.62951095e-09,  7.62951095e-09]),
 'EEG Fpz-O2': array([-4.09628443e-05,  5.00000000e-04,  5.00000000e-04, ...,
         7.62951095e-09,  7.62951095e-09,  7.62951095e-09]),
 'EEG Fpz-F7': array([5.71526665e-05, 2.18257420e-04, 3.29510948e-04, ...,
        7.62951095e-09, 7.62951095e-09, 7.62951095e-09]),
 'EEG F8-F7': array([ 2.44068055e-05, -5.00000000e-04, -5.00000000e-04, ...,
         7.62951095e-09,  7.62951095e-09,  7.62951095e-09]),
 'EEG F7-01': array([-1.38788434e-04, -5.00000000e-04, -5.00000000e-04, ...,
         7.62951095e-09,  7.62951095e-09,  7.62951095e-09]),
 'EEG F8-O2': array([-1.22522316e-04,  5.00000000e-04,  5.00000000e-04, ...,
         7.62951095e-09,  7.62951095e-09,  7.62951095e-09]),
 'EEG Fpz-F8': array([3.27382315e-05, 5.00000000e-04, 5.00000000e-04, ...,
        7.62951095e-09, 7.62951095e-09, 7.62951095e-09])}

In [8]:
from Ray.EEG_data import read_VG_file

# Understanding EEG_data_obj

# We'll call read_VG_file to create one such object.
# This requires a participant ID and a list of excluded channels
part_ID = 'VG_01'
exclude_channels = []
EEG_data_obj = read_VG_file(part_ID)

print(f"EEG_data_obj = {EEG_data_obj}")
print(f"list(EEG_data_obj.EEG_data.keys()) = {list(EEG_data_obj.EEG_data.keys())}")
channel_data = EEG_data_obj.EEG_data['EEG Fpz-O1']
print(f"type(channel_data) = {type(channel_data)}")
print(f"channel_data.shape = {channel_data.shape}")
print(f"channel_data[:10] = {channel_data[:10]}")


STUDY_DATA_FOLDER = ../data/CARE_HOME_DATA/
VG_file_path = ../data/CARE_HOME_DATA/./VG01/8921_15_52.edf
Extracting EDF parameters from /home/luke/git/external/predicament/data/CARE_HOME_DATA/VG01/8921_15_52.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
VG_file.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
include_channels=['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
type(data) = <class 'numpy.ndarray'>
data.shape = (13, 561000)
ch_name = EEG Fpz-O1
channel_lookup[ch_name] = 1
ch_name = EEG Fpz-O2
channel_lookup[ch_name] = 2
ch_name = EEG Fpz-F7
channel_lookup[ch_name] = 3
ch_name = EEG F8-F7
channel_lookup[ch_name] = 4
ch_name = EEG F7-01
channel_lookup[ch_name] = 5
ch_name = EEG F8-O2
channel_lookup[ch_name] = 6
ch_name = EEG

In [9]:
from predicament.utils.config import DREEM_EEG_CHANNELS
part_ID = 'VG_02'
include_channels = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'Positiongram', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
EEG_data_obj2 = read_VG_file(part_ID, include_channels = DREEM_EEG_CHANNELS)


STUDY_DATA_FOLDER = ../data/CARE_HOME_DATA/
VG_file_path = ../data/CARE_HOME_DATA/./VG02/7921_15_18.edf
Extracting EDF parameters from /home/luke/git/external/predicament/data/CARE_HOME_DATA/VG02/7921_15_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
VG_file.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
include_channels=['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
type(data) = <class 'numpy.ndarray'>
data.shape = (13, 541000)
ch_name = EEG Fpz-O1
channel_lookup[ch_name] = 1
ch_name = EEG Fpz-O2
channel_lookup[ch_name] = 2
ch_name = EEG Fpz-F7
channel_lookup[ch_name] = 3
ch_name = EEG F8-F7
channel_lookup[ch_name] = 4
ch_name = EEG F7-01
channel_lookup[ch_name] = 5
ch_name = EEG F8-O2
channel_lookup[ch_name] = 6
ch_name = EEG

In [10]:
for ch_name, data in EEG_data_obj2.EEG_data.items():
    print(f"Channel {ch_name} has {len(data)} items")

Channel EEG Fpz-O1 has 541000 items
Channel EEG Fpz-O2 has 541000 items
Channel EEG Fpz-F7 has 541000 items
Channel EEG F8-F7 has 541000 items
Channel EEG F7-01 has 541000 items
Channel EEG F8-O2 has 541000 items
Channel EEG Fpz-F8 has 541000 items


In [12]:
import pandas as pd
from Ray.Event_details import Event_time_details

event_details_path = '../data/CARE_HOME_DATA/event_details.csv'
exp_info_path = '../data/CARE_HOME_DATA/exp_info.csv'

event_details_df = pd.read_csv(event_details_path, index_col=0)
exp_info_df = pd.read_csv(exp_info_path, index_col=0)

part_list = list(exp_info_df.index)
ev_details = {}

for partID in part_list:
    ev_detail = Event_time_details(partID)
    ev_detail.set_exp_datetime(exp_info_df.loc[partID]['date'], exp_info_df.loc[partID]['exp_start'])    
    for event in event_details_df.loc[partID,].iterrows():
        start = str(event[1]['start']) if event[1]['start'] != 'None' else None
        end = str(event[1]['end']) if event[1]['end'] != 'None' else None
        print(f"partID: end = {partID}:{end}")
        ev_detail.set_event(event[1]['event'], start, end, event[1]['valid'])
    ev_details[partID] = ev_detail


partID: end = VG_01:None
(start, end) = ('15:52:00', None)
(start_timestamp, end_timestamp) = (1631112720.0, None)
partID: end = VG_01:None
(start, end) = ('15:52:00', None)
(start_timestamp, end_timestamp) = (1631112720.0, None)
partID: end = VG_01:16:10:00
(start, end) = ('16:02:00', '16:10:00')
(start_timestamp, end_timestamp) = (1631113320.0, 1631113800.0)
partID: end = VG_01:16:01:00
(start, end) = ('15:56:00', '16:01:00')
(start_timestamp, end_timestamp) = (1631112960.0, 1631113260.0)
partID: end = VG_01:16:21:00
(start, end) = ('16:16:00', '16:21:00')
(start_timestamp, end_timestamp) = (1631114160.0, 1631114460.0)
partID: end = VG_01:16:16:00
(start, end) = ('16:11:00', '16:16:00')
The event (tchaikovsky) is not in the event list. Please check!
partID: end = VG_01:16:27:00
(start, end) = ('16:22:00', '16:27:00')
(start_timestamp, end_timestamp) = (1631114520.0, 1631114820.0)
partID: end = VG_02:None
(start, end) = ('15:18:00', None)
(start_timestamp, end_timestamp) = (1631024280

In [13]:
partID = 'VG_01'
for event in event_details_df.loc[partID].iterrows():
    print(event)
float.nan

('VG_01', event       setup
start    15:52:00
end          None
valid        True
Name: VG_01, dtype: object)
('VG_01', event    baseline
start    15:52:00
end          None
valid        True
Name: VG_01, dtype: object)
('VG_01', event    exper_video
start       16:02:00
end         16:10:00
valid           True
Name: VG_01, dtype: object)
('VG_01', event    wildlife_video
start          15:56:00
end            16:01:00
valid              True
Name: VG_01, dtype: object)
('VG_01', event    familiar_music
start          16:16:00
end            16:21:00
valid              True
Name: VG_01, dtype: object)
('VG_01', event    tchaikovsky
start       16:11:00
end         16:16:00
valid           True
Name: VG_01, dtype: object)
('VG_01', event    family_inter
start        16:22:00
end          16:27:00
valid            True
Name: VG_01, dtype: object)


AttributeError: type object 'float' has no attribute 'nan'

In [14]:
import mne
VG_file_path = os.path.join(STUDY_DATA_FOLDER, VG_file_paths[part_ID])
mne_Raw_obj = mne.io.read_raw_edf(VG_file_path, exclude =[])
print(type(mne_Raw_obj))
print(f"mne_Raw_obj.ch_names = {mne_Raw_obj.ch_names}")
mne_Raw_obj.info

Extracting EDF parameters from /home/luke/git/external/predicament/data/CARE_HOME_DATA/VG02/7921_15_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<class 'mne.io.edf.edf.RawEDF'>
mne_Raw_obj.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']


<Info | 7 non-empty values
 bads: []
 ch_names: Accelero Norm, EEG Fpz-O1, EEG Fpz-O2, EEG Fpz-F7, EEG F8-F7, ...
 chs: 13 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: 2021-09-07 15:18:45 UTC
 nchan: 13
 projs: []
 sfreq: 250.0 Hz
>

In [15]:
to_pick = ['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
mne_Raw_obj.pick(to_pick)
print(f"mne_Raw_obj.ch_names = {mne_Raw_obj.ch_names}")

mne_Raw_obj

mne_Raw_obj.ch_names = ['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']


<RawEDF | 7921_15_18.edf, 7 x 541000 (2164.0 s), ~12 kB, data not loaded>

In [16]:
from Ray import EEG_data, E4_data
from predicament.utils.config import data_folder, VG_file_paths, VG_Hz, EEG_buffer
from Ray.Event_details import Event_time_details
from Ray.data_load_save import save_info_to_csv

save_info_to_csv()

ImportError: cannot import name 'data_folder' from 'predicament.utils.config' (/home/luke/git/external/predicament/predicament/utils/config.py)

In [17]:
# resulting output from following call is
# dictionary str to type EEG_data_obj
# where EEG_data_obj is defined in Ray.EEG_data

all_eeg_data = EEG_data.read_all_VG_files()
event_details_dic = {
    'partID': [],
    'event': [],
    'start': [],
    'end': [],
    'valid': []
}

exp_info_dic = {
    'partID': [],
    'date': [],
    'exp_start': []
}

STUDY_DATA_FOLDER = ../data/CARE_HOME_DATA/
VG_file_path = ../data/CARE_HOME_DATA/./VG01/8921_15_52.edf
Extracting EDF parameters from /home/luke/git/external/predicament/data/CARE_HOME_DATA/VG01/8921_15_52.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
VG_file.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
include_channels=['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
type(data) = <class 'numpy.ndarray'>
data.shape = (13, 561000)
ch_name = EEG Fpz-O1
channel_lookup[ch_name] = 1
ch_name = EEG Fpz-O2
channel_lookup[ch_name] = 2
ch_name = EEG Fpz-F7
channel_lookup[ch_name] = 3
ch_name = EEG F8-F7
channel_lookup[ch_name] = 4
ch_name = EEG F7-01
channel_lookup[ch_name] = 5
ch_name = EEG F8-O2
channel_lookup[ch_name] = 6
ch_name = EEG

EDF file detected
Setting channel info structure...
Creating raw.info structure...
VG_file.ch_names = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'Positiongram', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
include_channels=['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
type(data) = <class 'numpy.ndarray'>
data.shape = (14, 1074500)
ch_name = EEG Fpz-O1
channel_lookup[ch_name] = 1
ch_name = EEG Fpz-O2
channel_lookup[ch_name] = 2
ch_name = EEG Fpz-F7
channel_lookup[ch_name] = 3
ch_name = EEG F8-F7
channel_lookup[ch_name] = 4
ch_name = EEG F7-01
channel_lookup[ch_name] = 5
ch_name = EEG F8-O2
channel_lookup[ch_name] = 6
ch_name = EEG Fpz-F8
channel_lookup[ch_name] = 7
Successfully loaded VG_10 VG file (EEG data).
STUDY_DATA_FOLDER = ../data/CARE_HOME_DATA/
VG_file_path = ../data/CARE_HOME_DATA/./VH01/61021_11_17.edf
Extracting

In [18]:
all_eeg_data['VG_05'].event_details.events_info.items()

AttributeError: 'NoneType' object has no attribute 'events_info'

In [ ]:
event_ids = dict(familiar_music=1, wildlife_video=2, family_inter=3, tchaikovsky=4, exper_video=5)
event_ids['familiar_music']

In [ ]:
from Ray.data_load_save import set_up
EEG_files, _ = set_up(isE4 = False)

In [ ]:
EEG_files['VG_01'].event_details.events_info

## Train/test data (evaluation data)


The method `prepare_evaluation_data.py` loads the obervational and eeg data and integrates it to produce some train and test data in 'data/EEG-Motor-Movement-Imagery-Dataset/5classes_part/{train,test}_{set,labels}.csv' (at time of writing). Let's investigate this.


In [ ]:
import pandas as pd
test_set_fname = '../data/EEG-Motor-Movement-Imagery-Dataset/5classes_part/test_set.csv'

test_set_df = pd.read_csv(test_set_fname, header=None)

test_set_df

In [ ]:
4096/250

## Time-series data



In [ ]:
from Ray import data_load_save
EEG_files, E4_files = data_load_save.set_up(
    isEEG=True, isE4=False)
    # file_utils.gen_train_test_valid(EEG_files)

In [ ]:
import matplotlib.pyplot as plt
# Visualize VG data (EEG 250Hz)
EEG_partID, EEG_channel, EEG_event = 'VG_06', 'EEG Fpz-O1', 'familiar_music'
plt.plot(EEG_files[EEG_partID].get_EEG_by_channel_and_event(EEG_channel, EEG_event), label = EEG_channel)
plt.plot(EEG_files[EEG_partID].get_EEG_by_channel_and_event('EEG F7-01', EEG_event), label = 'F7-01')
plt.plot(EEG_files[EEG_partID].get_EEG_by_channel_and_event('EEG Fpz-F8', EEG_event), label = 'Fpz-F8')

## Study Events




In [ ]:
from predicament.utils.config import EVENT_DETAILS_PATH
from predicament.utils.config import EXP_INFO_PATH
event_details_path = EVENT_DETAILS_PATH
exp_info_path = EXP_INFO_PATH
print(f"event_details_path = {event_details_path}")
event_details_df = pd.read_csv(event_details_path, index_col=0)
print(f"event_details_df.loc['VG_01'] = {event_details_df.loc['VG_01']}")
exp_info_df = pd.read_csv(exp_info_path, index_col=0)

In [ ]:
event_details_df.loc['VG_01',]

In [ ]:
for row in event_details_df.loc['VG_01',].iterrows():
    print(row[1]['event'])